# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [24]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19


testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
# testPicture = load_img('../resizeddeerdownload.jpg')
# testPicture = load_img('../DeerNight.jpg')
# testPicture = load_img('../resizedDeerNight.jpg')


pictureArray = img_to_array(testPicture)
# reshape data for the model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# load the model
new_input = keras.Input(shape=(224, 224, 3))
model = VGG19()
# USE above for basic tests, below for adding to model
# model = VGG19(include_top=False, input_tensor=new_input)
model.trainable = False
# predict the probability across all output classes
yhat = model.predict(pictureArray)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = labels[0][0]
# print the classification
print(labels)
model.summary()

1/1 [==============================] - 0s 187ms/step
[[('n02423022', 'gazelle', 0.4982862), ('n02422699', 'impala', 0.4615827), ('n02422106', 'hartebeest', 0.033281878), ('n02417914', 'ibex', 0.0022351271), ('n02115913', 'dhole', 0.0011182867)]]
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
             

2023-09-20 21:30:45.586937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Try with a different input shape

In [17]:
testPicture = load_img('../DeerNight.jpg')
pictureArray = img_to_array(testPicture)
# print (pictureArray.shape)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))

pictureArray = preprocess_input(pictureArray)
# load model and specify a new input shape for images
new_input = keras.Input(shape=(2592, 4608, 3))
newModel = VGG19(include_top=False, input_tensor=new_input, pooling="maz")
newModel.trainable = False
pictureArray = preprocess_input(pictureArray)
newModel.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 2592, 4608, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 2592, 4608, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 2592, 4608, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 1296, 2304, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, 1296, 2304, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 1296, 2304, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 648, 1152, 128)    0     

## Create new Output layer

In [21]:
from keras.layers import Dense, Activation, Flatten

completeModel = models.Sequential()
completeModel.add(newModel)
completeModel.add(Flatten())
completeModel.add(Dense(100, activation = "relu"))
completeModel.add(Dense(2, activation = "softmax"))
completeModel.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 81, 144, 512)      20024384  
                                                                 
 flatten_6 (Flatten)         (None, 5971968)           0         
                                                                 
 dense_11 (Dense)            (None, 100)               597196900 
                                                                 
 dense_12 (Dense)            (None, 2)                 202       
                                                                 
Total params: 617,221,486
Trainable params: 597,197,102
Non-trainable params: 20,024,384
_________________________________________________________________


## Orgainze test and training data

In [ ]:
trainImages = 
trainCatagories = 
testImages = 
testCatagories = 

## Train Last three layers

In [ ]:
completeModel.fit(trainImages, trainCatagories, epochs=25, batch_size=50)

## Test Some data

In [ ]:
score = completeModel.evaluate(testImages, testCatagories, verbose = 0)